In [7]:
import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from hpsklearn import HyperoptEstimator , tfidf, pca, any_text_preprocessing
from numpy import set_printoptions
from hyperopt import hp, tpe, fmin
from hyperopt import STATUS_OK, Trials
from hyperopt import space_eval
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import accuracy_score



import string
import collections
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


 
BILL_PROGRESS = {}
BILL_PROGRESS[1] = "Introduced"
BILL_PROGRESS[2] = "Engrossed"
BILL_PROGRESS[3] = "Enrolled"
BILL_PROGRESS[4] = "Passed"
BILL_PROGRESS[5] = "Vetoed"
BILL_PROGRESS[6] = "Failed"
BILL_PROGRESS[7] = "Override"
BILL_PROGRESS[8] = "Chaptered"
BILL_PROGRESS[9] = "Refer"
BILL_PROGRESS[10] = "Report Pass"
BILL_PROGRESS[11] = "Report DNP"
BILL_PROGRESS[12] = "Draft"
"""
1 Introduced
2 Engrossed
3 Enrolled
4 Passed
5 Vetoed
6 Failed Limited support based on state
7 Override Progress array only
8 Chaptered Progress array only
9 Refer / Progress array only
10 Report Pass / Progress array only
11 Report DNP / Progress array only
12 Draft Progress / array only
"""

POLITICAL_PARTY = {}
POLITICAL_PARTY[1] = "Democrat"
POLITICAL_PARTY[2] = "Republican"
POLITICAL_PARTY[3] = "Independent"
POLITICAL_PARTY[4] = "Green Party"
POLITICAL_PARTY[5] = "Libertarian"
POLITICAL_PARTY[6] = "Nonpartisan"


TEXT_TYPES = {}
TEXT_TYPES[1] = "Introduced"
TEXT_TYPES[2] = "Committee Substitute"
TEXT_TYPES[3] = "Amended"
TEXT_TYPES[4] = "Engrossed"
TEXT_TYPES[5] = "Enrolled"
TEXT_TYPES[6] = "Chaptered"
TEXT_TYPES[7] = "Fiscal Note"
TEXT_TYPES[8] = "Analysis"
TEXT_TYPES[9] = "Draft"
TEXT_TYPES[10] = "Conference Substitu"
TEXT_TYPES[11] = "Prefiled"
TEXT_TYPES[12] = "Veto Message"
TEXT_TYPES[13] = "Veto Response"
TEXT_TYPES[14] = "Substitute"

BILL_TYPE = {}
BILL_TYPE[1] = "Bill"
BILL_TYPE[2] = "Resolution"
BILL_TYPE[3] = "Concurrent Resolution"
BILL_TYPE[4] = "Joint Resolution"
BILL_TYPE[5] = "Joint Resolution Constitutional Amendment"
BILL_TYPE[6] = "Executive Order"
BILL_TYPE[7] = "Constitutional Amendment"
BILL_TYPE[8] = "Memorial"
BILL_TYPE[9] = "Claim"
BILL_TYPE[10] = "Commendation"
BILL_TYPE[11] = "Committee Study Request"
BILL_TYPE[12] = "Joint Memorial"
BILL_TYPE[13] = "Proclamation"
BILL_TYPE[14] = "Study Request"
BILL_TYPE[15] = "Address"
BILL_TYPE[16] = "Concurrent Memorial"
BILL_TYPE[17] = "Initiative"
BILL_TYPE[18] = "Petition"
BILL_TYPE[19] = "Study Bill"
BILL_TYPE[20] = "Initiative Petition"
BILL_TYPE[21] = "Repeal Bill"
BILL_TYPE[22] = "Remonstration"
BILL_TYPE[23] = "Committee Bill"


"""
1 B Bill
2 R Resolution
3 CR Concurrent Resolution
4 JR Joint Resolution
5 JRCA Joint Resolution Constitutional Amendment
6 EO Executive Order
7 CA Constitutional Amendment
8 M Memorial
9 CL Claim
10 C Commendation
11 CSR Committee Study Request
12 JM Joint Memorial
13 P Proclamation
14 SR Study Request
15 A Address
16 CM Concurrent Memorial
17 I Initiative
18 PET Petition
19 SB Study Bill
20 IP Initiative Petition
21 RB Repeal Bill
22 RM Remonstration
23 CB Committee Bill
"""
VOTES_TYPE = {}

VOTES_TYPE[1] = "Yea"
VOTES_TYPE[2] = "Nay"
VOTES_TYPE[3] = "Not Voting / Abstain"
VOTES_TYPE[4] = "Absent / Excused"


MIME_TYPES = {}

MIME_TYPES[1] = "HTML"
MIME_TYPES[2] = "PDF"
MIME_TYPES[3] = "WordPerfect"
MIME_TYPES[4] = "MS Word (.doc)"
MIME_TYPES[5] = "Rich Text Format"
MIME_TYPES[6] = "MS word 2007 (.docx)"

    
with open('NH_bill_dict','rb') as f:
    bill_data = pickle.load(f)
with open('NH_vote_dict','rb') as f:
    vote_data = pickle.load(f)
with open('NH_people_dict','rb') as f:
    people_data = pickle.load(f)
with open('NH_people_dict','rb') as f:
    people_data = pickle.load(f)
with open('NHlegoverall.dict','rb') as f:
    leg_data = pickle.load(f)


all_bills_by_id = {}
    
all_bills = {}
all_bills['bills'] = []
c = 0
for key in bill_data.keys():
    for k in bill_data[key]:
        all_bills['bills'].append(k)
        all_bills_by_id[k['bill_id']] = k['bill_id']


all_votes = {}
all_votes['votes'] = []

for key in vote_data.keys():
    for k in vote_data[key]:
        all_votes['votes'].append(k)

from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() 
                            if len(locs)>1)


people_hash = {}
        
all_people = {}
all_people['people'] = []

for key in people_data.keys():
    for k in people_data[key]:
        all_people['people'].append(k)
        if not int(k['people_id']) in people_hash:
            people_hash[int(k['people_id'])] = int(k['party_id'])
        
#print(len(all_people['people']))
#print("hashtable" ,len(people_hash))

print("before", len(all_people['people']))
dups = []
party_dups = []
final_list = [] 
for num in all_people['people']: 
    if num['people_id'] not in final_list: 
        final_list.append(num['people_id'])
    else:
        dups.append(num['people_id'])
        party_dups.append(num['party'])
        
for dup in sorted(list_duplicates(dups)):
    tempy = []
    for dd in dup[1]:
        tempy.append(party_dups[dd])
    #print(dup[0], tempy)

        

"""
visited_index = []
visited = []
for i, val in enumerate(all_people['people']):
    if not int(val['people_id']) in visited:
        print(int(val['people_id']), "->", visited[visited.index(int(val['people_id']))])
        print(val['party'], "->", all_people['people'][visited.index(int(val['people_id']))]['party'])
    visited.append(val['people_id'])
    visited_index.append(i)
"""






bill_by_id = {}

for key in bill_data.keys():
    if ((key == 2017) or (key == 2018) or (key == 2019)):
        for k in bill_data[key]:
            bill_by_id[int(k['bill_id'])] = k

total = 0
votes_by_bill = {}
bill_votes = {}
bill_votes['v'] = []
for key in vote_data.keys():
    if ((key == 2017) or (key == 2018) or (key == 2019)):
        for k in vote_data[key]:
            total += len(k['votes'])
            bill_votes['v'].append(k)
            if int(k['bill_id']) in votes_by_bill:
                votes_by_bill[int(k['bill_id'])].append(k)
            else:
                votes_by_bill[int(k['bill_id'])] = [k]

            
print(len(bill_votes['v']))

print(len(votes_by_bill))

print(len(bill_by_id))

print(total)




with open('/home/ec2-user/SageMaker/ProjectQuigley/people_ideology_table.dict', 'rb') as f:
    politician_leadership = pickle.load(f) 
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_leadership_table.dict', 'rb') as f:
    politician_ideology = pickle.load(f)

with open('NHlegoverallLabel.list','rb') as f:
    leg_target = pickle.load(f)    






ModuleNotFoundError: No module named 'hpsklearn'

In [12]:
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_ideology.dict', 'rb') as f:
    ideology = pickle.load(f) 
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_leadership.dict', 'rb') as f:
    leadership = pickle.load(f)
    
print(ideology["2015-2016-2017-2018-2019"])

{11759: 1.0, 4986: 0.9919837907600131, 16826: 0.891673876184671, 12836: 0.8453931938782167, 5092: 0.8363246758015632, 19839: 0.8224557635921402, 4772: 0.8222574005796075, 4805: 0.8061478901913844, 5033: 0.7827680244957079, 4956: 0.7816162140034678, 5065: 0.7760402869949445, 10471: 0.7756532286933506, 17487: 0.7426720051199148, 15558: 0.7288237550770044, 5006: 0.7227823192362455, 5015: 0.7210422104521186, 4853: 0.709948097984068, 4860: 0.7059621959210816, 5082: 0.7014954817457552, 12822: 0.6995629281226461, 4949: 0.6890897823510626, 15531: 0.6879030410100981, 10455: 0.6875231150669416, 18577: 0.6844637241240348, 5080: 0.6803507983508148, 4926: 0.678290457699569, 11989: 0.6776937600001456, 17100: 0.6750583948160161, 4896: 0.6679095015724439, 10480: 0.6653070435563476, 17477: 0.6652966504698207, 10470: 0.6647580723114604, 18170: 0.6644338989924531, 15605: 0.6638847563862497, 16397: 0.6634150948733093, 15324: 0.6598238059208141, 10490: 0.6585339463338079, 10462: 0.657565949581472, 4802: 0.

In [5]:
#define a performance metrics function (accuracy and percision and recall)
def perf_metric (y_test, y_pred):
    length = len(y_test)
#Accuracy
    x=0
    y=0
    acc=0
    for i in range(length):
        if y_test[i] == y_pred[i]:
            x+=1
        y+=1
    acc=x/y*100
    print(x,y,len(y_pred),len(y_test))
    print('Accuracy:', acc)

#Precision and Recall
    tp=0
    fp=0
    fn=0
    tn=0
    perc=0 
    recall= 0 
        
    for i in range(length):
        if y_pred[i]==1 and y_test[i]==1:
            tp+=1
        elif y_pred[i]==1 and y_test[i]==0:
            fp+=1
        elif y_pred[i]==0 and y_test[i]==1:
            fn+=1
        elif y_pred[i]==0 and y_test[i]==0:
            tn+=1
        
    if tp==0 and fp==0: 
        perc=0
    else:
        perc= tp/(tp+fp)  
          
    if tp==0 and fn==0: 
        recall=0
    else: 
        recall = tp/(tp+fn)

#print(tp,fp,fn,tn)
    print('Precision:', perc)
    print('Recall', recall)


In [12]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [13]:


bill_data_17_19 = {}
bill_data_17_19['bills'] = []

vote_data_17_19 = {}
vote_data_17_19['votes'] = []

passed_bills = []
vote_bill = []

for key in bill_data.keys():

    if key == 2017:
        print(key, len(bill_data[key]))
        for b in bill_data[key]:
            size = 0
            prog = b['progress']
            for p in prog:
                if p['event'] == 4:
                    passed_bills.append(b['bill_id'])
    elif key == 2018:
        print(key, len(bill_data[key]))
        for b in bill_data[key]:
            size = 0
            prog = b['progress']
            for p in prog:
                if p['event'] == 4:
                    passed_bills.append(b['bill_id'])

            

counter = 0
for key in bill_data.keys():
    if key == 2017 or 2018:
        for b in bill_data[key]:
            if b['completed'] == 1:
                counter += 1
print(counter)
print("bills passed from 2017-2019", len(passed_bills))
            
for b in passed_bills:
    if b == 1049105:
        print("TRUE")
            
for key in bill_data.keys():
    if key == 2017 or 2018 or 2019:
        for ele in bill_data[key]:
            if ele['votes'] == []:
                continue
            else:
                res = ele['votes']
                for r in res:
                    vote_bill.append(r['roll_call_id'])
            

                    
#print(len(vote_bill))
poli_data = {}
poli_data['poli'] = []
#should be 1941 elements
for key in vote_data.keys():
    year = vote_data[key]
    for temp in year:
        if temp['roll_call_id'] in vote_bill:
            #print(temp['roll_call_id'])
            res = temp
            #print(res['bill_id'])
            val = {}
            val['bill_id'] = res['bill_id']
            val['roll_call_id'] = res['roll_call_id']
            val['date'] = res['date']
            val['yea'] = res['yea']
            val['nay'] = res['nay']
            val['nv'] = res['nv']
            val['absent'] = res['absent']
            val['total'] = res['total']
            val['passed'] = res['passed']
            val['chamber'] = res['chamber']
            val['votes'] = res['votes']
            poli_data['poli'].append(val)




2017 871
2018 1111
752
bills passed from 2017-2019 651
TRUE


In [208]:
leg_data = pd.read_pickle("NHlegclusters.df")

bill_cat = {}

print(leg_data.info())

for i, val in leg_data.iterrows():
    #temp = str(val['bill_cat'])
    bill_cat[val['doc_id']] = int(val['cluster'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4060 entries, 0 to 4059
Data columns (total 2 columns):
cluster    4060 non-null int64
doc_id     4060 non-null int64
dtypes: int64(2)
memory usage: 63.6 KB
None


In [14]:
bills = pd.read_pickle("embedded_pol.DF")
leg_data = pd.read_csv("leg_data.csv")
num_vects = 16

data = {}


data['bill_id'] = []
data['bill_cat'] = []
#data['bill_type_id'] = []
data['roll_call_id'] = []
data['passed'] = []
data['chamber_id'] = []
data['people_id'] = []
data['party_id'] = []
#need to fill in
#data['ideology'] = []
#data['leadership']=[]

data['doc_id'] = []
for i in range(1, num_vects+1):
    data['dv{}'.format(i)] = []


data_2 = {}
data_2['people_id'] = []
#data_2['party_id'] = []
#data_2['bill_type_id'] = []



sample_bills = []

client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_people_data']

target = []
#print(bills.loc[bills['bill_id'] == 895290])
for i, val in bills.iterrows():
    
    if int(val['bill_id']) in votes_by_bill:
        
        for v in votes_by_bill[int(val['bill_id'])]:
            sample_bills.append(int(v['bill_id']))

            for vv in v['votes']:
                data['bill_id'].append(int(val['bill_id']))
                
                data['bill_cat'].append(bill_cat[val['doc_id']])
                #data['bill_type_id'].append(int(bill_by_id[int(val['bill_id'])]['bill_type_id']))
                data['roll_call_id'].append(int(v['roll_call_id']))
                data['passed'].append(int(v['passed']))
                data['chamber_id'].append(int(v['chamber_id']))
                data['people_id'].append(int(vv['people_id']))
                data['party_id'].append(int(people_hash[int(vv['people_id'])] if int(vv['people_id']) in people_hash else -1))
                try:
                    data['ideology'].append(ideology["2015-2016-2017-2018-2019"][vv['people_id']])
                except:
                    data['ideology'].append(0.5)
                try:
                    data['leadership'].append(leadership["2015-2016-2017-2018-2019"])
                except:
                    data['leadership'].append(0)

                
                data['doc_id'].append(int(val['doc_id']))
                for i in range(1, num_vects+1):
                    data['dv{}'.format(i)].append(val['dv{}'.format(i)])
                target.append(int(vv['vote_id']) if int(vv['vote_id']) == 1 else 0)
                #target.append(int(vv['vote_id']))
                #data_2['people_id'].append(int(vv['people_id']))
                #data_2['party_id'].append(int(people_hash[int(vv['people_id'])] if int(vv['people_id']) in people_hash else -1))
                #data_2['bill_type_id'].append(int(bill_by_id[int(val['bill_id'])]['bill_type_id']))
                

poli_df = pd.DataFrame(data)
print(poli_df)
#poli_df_2 = pd.DataFrame(data_2)


"""
#pd.to_numeric(poli_df['bill_id'], errors='coerce').fillna(0)
result_1 = bills.join(poli_df.set_index('bill_id'), on='bill_id')
print("1", result_1)
result_2 = poli_df.merge(bills, on='bill_id')
print("2", result_2)
result_3 = pd.merge(bills, poli_df, on='bill_id')
print("3", result_3)
result_4 = pd.merge(poli_df, bills, on='bill_id')
print("4", result_4)
"""

full_data = {}
full_data = poli_df.copy()
full_data["vote_id"] = target


df_samples = pd.DataFrame(sample_bills)

#print("result", result.loc[result['bill_id'] == 895290])


                    
                

NameError: name 'pymongo' is not defined

In [224]:

#rslt_df = dataframe[dataframe['Stream'].isin(options)]

data_house = {}
data_house = full_data.copy()
data_house_target = []

house = data_house.copy()


house_test = house.loc[(house['chamber_id'] == 67) & (house['roll_call_id'].isin(sampled))]

house_train = house.drop(sampled)

house_train
house_test



for i, val in data_house.iterrows():
    data_house_target.append(int(val['vote_id']))
data_house = data_house.drop(columns=['vote_id'])
print(data_house)
house_result = pd.get_dummies(data_house)
house_target = pd.DataFrame(data_house_target)

X_train, X_test, y_train, y_test = train_test_split(house_result, house_target, test_size=0.2, random_state=13, shuffle=False) 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

clf = RandomForestClassifier(
    min_samples_split=5, 
    criterion = 'gini', 
    max_features= 0.4, 
    max_depth= 30)

clf = RandomForestClassifier()
clf.fit(X_train, y_train.ravel())  
y_pred= clf.predict(X_test)
perf_metric(y_test, y_pred)


model = ExtraTreesClassifier(n_estimators=15)
model.fit(X_train, y_train.ravel())


feats = model.feature_importances_

cols = list(data_house.columns.values)


for i, ele in enumerate(feats):
    print(cols[i], ele*1000)
#print("poli", poli_df.loc[poli_df['bill_id'] == '1162256'])

pca = PCA(n_components=5)
fit = pca.fit(X_train)
# summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)


        bill_cat  chamber_id   doc_id       dv1      dv10      dv11      dv12  \
0             34          67  1441853  0.532009 -0.796759  1.197310  1.540120   
1             34          67  1441853  0.532009 -0.796759  1.197310  1.540120   
2             34          67  1441853  0.532009 -0.796759  1.197310  1.540120   
3             34          67  1441853  0.532009 -0.796759  1.197310  1.540120   
4             34          67  1441853  0.532009 -0.796759  1.197310  1.540120   
...          ...         ...      ...       ...       ...       ...       ...   
413733        20          68  1860514  1.283047  1.733561  0.400574  0.369438   
413734        20          68  1860514  1.283047  1.733561  0.400574  0.369438   
413735        20          68  1860514  1.283047  1.733561  0.400574  0.369438   
413736        20          68  1860514  1.283047  1.733561  0.400574  0.369438   
413737        20          68  1860514  1.283047  1.733561  0.400574  0.369438   

            dv13      dv14 

In [45]:

data_senate = {}
data_senate = full_data.copy()

#print(bill_by_id[894796]['bill_type_id'])
#print(poli_df)
#print(full_data)


data_senate_target = []
data_senate = data_senate.loc[full_data['chamber_id'] == 68]
for i, val in data_senate.iterrows():
    data_senate_target.append(int(val['vote_id']))
data_senate = data_senate.drop(columns=['vote_id'])
senate_result = pd.get_dummies(data_senate)
senate_target = pd.DataFrame(data_senate_target)

X_train, X_test, y_train, y_test = train_test_split(senate_result, senate_target, test_size=0.3, random_state=13, shuffle=False) 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

clf = RandomForestClassifier(
    min_samples_split=5, 
    criterion = 'gini', 
    max_features= 0.4, 
    max_depth= 30)

clf = RandomForestClassifier()
clf.fit(X_train, y_train.ravel())  
y_pred= clf.predict(X_test)
perf_metric(y_test, y_pred)



# feature extraction
model = ExtraTreesClassifier(n_estimators=15)
model.fit(X_train, y_train.ravel())
feats = model.feature_importances_

print("\n\n")

cols = list(data_senate.columns.values)
for i, ele in enumerate(feats):
    print(cols[i], ele*1000)

    
pca = PCA(n_components = 'mle', svd_solver = 'full')
fit = pca.fit(X_train)
# summarize components

print("\n\nExplained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)


model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 8)
fit = rfe.fit(X_train, y_train.ravel())
print("\n\nNum Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)
feats = fit.ranking_

for i, ele in enumerate(feats):
    print(cols[i], ele)



28509 36946 36946 36946
Accuracy: 77.16396903589022
Precision: 0.7730232558139535
Recall 0.8237101650394013



bill_id 16.108460913921373
bill_type_id 0.024795399225934336
chamber_id 0.0
doc_id 5.687688951254773
dv1 6.114995532969683
dv10 5.71448486503493
dv11 9.033955339415082
dv12 7.613536479951915
dv13 6.023034413473595
dv14 6.141024462299823
dv15 9.340686186721078
dv16 5.8176107555007786
dv2 5.970168896945244
dv3 5.110276833978965
dv4 6.727137748768767
dv5 8.53800198740376
dv6 5.031106384068578
dv7 8.907147693209243
dv8 7.725537525161035
dv9 6.441469392808808
party_id 285.79556702829126
passed 52.28145098944312
people_id 487.5987288409573
roll_call_id 42.253133379195


Explained Variance: [9.345e-01 4.925e-02 1.471e-02 1.506e-03 6.721e-11 4.927e-11 4.699e-11
 4.061e-11 3.764e-11 2.807e-11 2.622e-11 2.392e-11 2.102e-11 1.694e-11
 1.618e-11 1.602e-11 1.372e-11 1.325e-11 8.360e-12 7.394e-12 6.918e-12
 3.495e-12 9.665e-13]
[[-4.122e-01  6.479e-08 -1.790e-19 -8.178e-01 -1.393e-07 -8.317

In [34]:

result = pd.get_dummies(poli_df)
target = pd.DataFrame(target)

target = pd.get_dummies(target)
X_train, X_test, y_train, y_test = train_test_split(result, target, test_size=0.3, random_state=13, shuffle=False) 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

clf = RandomForestClassifier(
    min_samples_split=5, 
    criterion = 'gini', 
    max_features= 0.4, 
    max_depth= 30)

#y_pred=grid_search.pred(X_test)
#y_pred=grid_search.pred(X_test)


clf = RandomForestClassifier()
clf.fit(X_train, y_train.ravel())  
y_pred= clf.predict(X_test)

perf_metric(y_test, y_pred)


96929 124122 124122 124122
Accuracy: 78.091716214692
Precision: 0.7874483584181253
Recall 0.8102412353170908


In [40]:
clf=GaussianNB()

estim = HyperoptEstimator(classifier=clf, max_evals=30)
estim.fit(X_train, y_train.ravel())#X_train.values, y_train
#clf.fit(X_train,y_train)
y_pred = estim.predict(X_test)
perf_metric(y_test,y_pred)

100%|██████████| 1/1 [00:00<00:00,  1.52it/s, best loss: 0.0]
124122 124122 124122 124122
Accuracy: 100.0
Precision: 1.0
Recall 1.0


In [18]:
clf = MLPClassifier()
estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest, max_evals=2)
estim.fit(X_train, y_train.ravel())
y_pred = estim.predict(X_test)
perf_metric(y_test, y_pred)

100%|██████████| 1/1 [00:08<00:00,  8.79s/it, best loss: 0.43985222015054204]
66574 124122 124122 124122
Accuracy: 53.63593883437263
Precision: 1.0
Recall 1.0


In [26]:
clf = RandomForestClassifier(
    min_samples_split=5, 
    criterion = 'gini', 
    max_features= 0.4, 
    max_depth= 30)

clf = RandomForestClassifier()
clf.fit(X_train, y_train.ravel())  
y_pred= clf.predict(X_test)

perf_metric(y_test, y_pred)

92920 124122 124122 124122
Accuracy: 74.86182949034014
Precision: 0.7326554237154358
Recall 0.8365878571213987


In [27]:
clf = DecisionTreeClassifier()
#  preprocessing= any_text_preprocessing
estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest)
estim.fit(X_train, y_train)#X_train.values, y_train
#clf.fit(X_train,y_train)
y_pred = estim.predict(X_test)
perf_metric(y_test, y_pred)

100%|██████████| 1/1 [00:02<00:00,  2.65s/it, best loss: 0.4577895172985291]
87654 124122 124122 124122
Accuracy: 70.6192294677817
Precision: 0.7030074173971679
Recall 0.7830083816504941


In [88]:
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()



In [153]:
test_data = full_data.loc[full_data['people_id'] == 4753]

#1159319 1161861    1162256

tempy = test_data.copy()

print(tempy)

print("testinging\n\n\n")
_temp = []
new_data = {}
new_data = tempy.loc[tempy['bill_id'] == 957465]
for i in new_data.iterrows(): 
    for j in range(1, num_vects+1):
        _temp.append(float(val['dv{}'.format(j)]))
    break

x = np.array(_temp)
xx = x.reshape(1, 16)

print(x)

final['bill_id'] = []
final['roll_call_id'] = []
final['cos_sim'] = []
final['vote_id'] = []
                         
visited = [957465]
for i, val in tempy.iterrows():
    tempy = []
    if val['bill_id'] in visited:
        continue
    else:
        for j in range(1, num_vects+1):
            tempy.append(float(val['dv{}'.format(j)]))
        y = np.array(tempy)
        yy = y.reshape(1, 16)
        #print("tem", y)
        cos_sim = cosine_similarity(xx, yy)
        final['bill_id'].append(int(val['bill_id']))
        final['roll_call_id'].append(int(val['roll_call_id']))
        final['cos_sim'].append(cos_sim[0][0])
        final['vote_id'].append(int(val['vote_id']))
        visited.append(int(val['bill_id']))
final = pd.DataFrame(final)

total = 0



total = sum(final['cos_sim']*final['vote_id'])


print(total / 434)




    


        bill_id  chamber_id   doc_id       dv1      dv10      dv11      dv12  \
4        894796          67  1441853  0.532009 -0.796759  1.197310  1.540120   
446      957465          67  1509871  0.923941  1.427340  0.375213  0.975882   
840      957465          67  1509871  0.923941  1.427340  0.375213  0.975882   
1234     957465          67  1509871  0.923941  1.427340  0.375213  0.975882   
1652     894761          67  1441820  0.246843 -0.206302  1.031898 -0.269038   
...         ...         ...      ...       ...       ...       ...       ...   
411504  1159319          68  1968725 -0.875881  0.726629  0.890007 -1.152275   
411976  1159319          68  2026490 -0.660196  0.519284  0.601249 -0.777388   
412520  1161861          68  1860172  1.283490  0.231333  0.969739 -0.153248   
412918  1161861          68  2033756  1.734223  0.277812  1.148614 -0.266285   
413340  1162256          68  1860514  1.283047  1.733561  0.400574  0.369438   

            dv13      dv14      dv15  .

ValueError: Length of values does not match length of index

In [174]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=15)
neigh.fit(X_train, y_train.ravel())
xx=X_test.reshape(len(X_test), 23)
y_pred = neigh.predict(xx)
perf_metric(y_test, y_pred)


#print(neigh.predict_proba([[0.9]]))

156374 290354 290354 290354
Accuracy: 53.856327104155625
Precision: 0.5414610125625551
Recall 0.6369576053131442


In [167]:
print(xx[15])

[ 9.575e+05  6.700e+01  1.510e+06  9.239e-01  1.427e+00  3.752e-01
  9.759e-01  7.850e-01  3.443e-01 -7.571e-01 -4.559e-01 -3.224e-02
 -8.857e-01 -7.390e-02  9.898e-01  1.706e-01 -3.010e-02  1.054e-01
 -1.292e+00  1.000e+00  0.000e+00  4.809e+03  6.187e+05]
